## Dimensionality Reduction


 For example,
consider the MNIST images (introduced in Chapter 3): the pixels on the image bor‐
ders are almost always white, so you could completely drop these pixels from the
training set without losing much information. 

Moreover, two neighboring pixels
are often highly correlated: if you merge them into a single pixel (e.g., by taking the
mean of the two pixel intensities), you will not lose much information.

you should first try to train
your system with the original data before considering using dimen‐
sionality reduction if training is too slow. In some cases, however,
reducing the dimensionality of the training data may filter out
some noise and unnecessary details and thus result in higher per‐
formance (but in general it won’t; it will just speed up training).

## The Curse of Dimensionality


making predictions much less relia‐
ble than in lower dimensions, since they will be based on much larger extrapolations.
In short, the more dimensions the training set has, the greater the risk of overfitting
it.

In theory, one solution to the curse of dimensionality could be to increase the size of
the training set to reach a sufficient density of training instances. Unfortunately, in
practice, the number of training instances required to reach a given density grows
exponentially with the number of dimensions.

## Main Approaches for Dimensionality Reduction


### Projection

In most real-world problems, training instances are not spread out uniformly across
all dimensions. Many features are almost constant, while others are highly correlated

### Manifold Learning


Many dimensionality reduction algorithms work by modeling the manifold on which
the training instances lie; this is called Manifold Learning. It relies on the manifold
assumption, also called the manifold hypothesis, which holds that most real-world
high-dimensional datasets lie close to a much lower-dimensional manifold. 

 In other words, the degrees
of freedom available to you if you try to create a digit image are dramatically lower
than the degrees of freedom you would have if you were allowed to generate any
image you wanted. These constraints tend to squeeze the dataset into a lowerdimensional manifold.

In short, if you reduce the dimensionality of your training set before training a
model, it will definitely speed up training, but it may not always lead to a better or
simpler solution; it all depends on the dataset.


### PCA

Principal Component Analysis (PCA) is by far the most popular dimensionality reduc‐
tion algorithm. First it identifies the hyperplane that lies closest to the data, and then
it projects the data onto it.

### Preserving the Variance


Before you can project the training set onto a lower-dimensional hyperplane, you
first need to choose the right hyperplane. 

 the projection onto the solid line preserves the maximum
variance, while the projection onto the dotted line preserves very little variance, and
the projection onto the dashed line preserves an intermediate amount of variance.


It seems reasonable to select the axis that preserves the maximum amount of var‐
iance, as it will most likely lose less information than the other projections.

### Principal Components


PCA identifies the axis that accounts for the largest amount of variance in the train‐
ing set. In Figure 8-7, it is the solid line. It also finds a second axis, orthogonal to the
first one, that accounts for the largest amount of remaining variance. 

The unit vector that defines the ith axis is called the ith principal component (PC).

PCA assumes that the dataset is centered around the origin.

### Projecting Down to d Dimensions


 Selecting this hyperplane ensures that the
projection will preserve as much variance as possible. 



In [ ]:
W2 = V.T[:, :2]
X2D = X_centered.dot(W2)

### Using Scikit-Learn


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
X2D = pca.fit_transform(X)

### Explained Variance Ratio


Another very useful piece of information is the explained variance ratio of each prin‐
cipal component, available via the explained_variance_ratio_ variable. 

In [ ]:
print(pca.explained_variance_ratio_)

This tells you that 84.2% of the dataset’s variance lies along the first axis, and 14.6%
lies along the second axis. This leaves less than 1.2% for the third axis, so it is reason‐
able to assume that it probably carries little information.


### Choosing the Right Number of Dimensions


Unless, of course, you are reducing dimen‐
sionality for data visualization—in that case you will generally want to reduce the
dimensionality down to 2 or 3.

In [ ]:
pca = PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1


You could then set n_components=d and run PCA again. However, there is a much
better option: instead of specifying the number of principal components you want to
preserve, you can set n_components to be a float between 0.0 and 1.0, indicating the
ratio of variance you wish to preserve:

In [ ]:
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X)

Yet another option is to plot the explained variance as a function of the number of
dimensions (simply plot cumsum; see Figure 8-8). There will usually be an elbow in the
curve, where the explained variance stops growing fast. You can think of this as the
intrinsic dimensionality of the dataset. In this case, you can see that reducing the
dimensionality down to about 100 dimensions wouldn’t lose too much explained var‐
iance.

### PCA for Compression


Obviously after dimensionality reduction, the training set takes up much less space.
For example, try applying PCA to the MNIST dataset while preserving 95% of its var‐
iance. You should find that each instance will have just over 150 features, instead of
the original 784 features. So while most of the variance is preserved, the dataset is
now less than 20% of its original size! This is a reasonable compression ratio, and you
can see how this can speed up a classification algorithm (such as an SVM classifier)
tremendously.

In [ ]:
pca = PCA(n_components = 154)
X_mnist_reduced = pca.fit_transform(X_mnist)
X_mnist_recovered = pca.inverse_transform(X_mnist_reduced)

### Incremental PCA


One problem with the preceding implementation of PCA is that it requires the whole
training set to fit in memory in order for the SVD algorithm to run. Fortunately,
Incremental PCA (IPCA) algorithms have been developed: you can split the training
set into mini-batches and feed an IPCA algorithm one mini-batch at a time. This is
useful for large training sets, and also to apply PCA online

In [ ]:
from sklearn.decomposition import IncrementalPCA
n_batches = 100
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_mnist, n_batches):
 inc_pca.partial_fit(X_batch)
X_mnist_reduced = inc_pca.transform(X_mnist)

In [ ]:
X_mm = np.memmap(filename, dtype="float32", mode="readonly", shape=(m, n))
batch_size = m // n_batches
inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mm)


### Randomized PCA


Scikit-Learn offers yet another option to perform PCA, called Randomized PCA. This
is a stochastic algorithm that quickly finds an approximation of the first d principal
components. 

In [ ]:
rnd_pca = PCA(n_components=154, svd_solver="randomized")
X_reduced = rnd_pca.fit_transform(X_mnist)

### Kernel PCA


It turns out that the same trick can be applied to PCA, making it possible to perform
complex nonlinear projections for dimensionality reduction.

In [ ]:
from sklearn.decomposition import KernelPCA
rbf_pca = KernelPCA(n_components = 2, kernel="rbf", gamma=0.04)
X_reduced = rbf_pca.fit_transform(X)

### Selecting a Kernel and Tuning Hyperparameters


kPCA is an unsupervised learning algorithm, there is no obvious performance
measure to help you select the best kernel and hyperparameter values. 

 For example, the following
code creates a two-step pipeline, first reducing dimensionality to two dimensions
using kPCA, then applying Logistic Regression for classification. Then it uses Grid
SearchCV to find the best kernel and gamma value for kPCA in order to get the best
classification accuracy at the end of the pipeline:

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
clf = Pipeline([
 ("kpca", KernelPCA(n_components=2)),
 ("log_reg", LogisticRegression())
 ])
param_grid = [{
 "kpca__gamma": np.linspace(0.03, 0.05, 10),
 "kpca__kernel": ["rbf", "sigmoid"]
 }]
grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X, y)

print(grid_search.best_params_)

 Fortunately, it is pos‐
sible to find a point in the original space that would map close to the reconstructed
point. This is called the reconstruction pre-image. Once you have this pre-image, you
can measure its squared distance to the original instance. You can then select the ker‐
nel and hyperparameters that minimize this reconstruction pre-image error.


In [ ]:
rbf_pca = KernelPCA(n_components = 2, kernel="rbf", gamma=0.0433,
 fit_inverse_transform=True)
X_reduced = rbf_pca.fit_transform(X)
X_preimage = rbf_pca.inverse_transform(X_reduced)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(X, X_preimage)


## LLE

Locally Linear Embedding (LLE)8
 is another very powerful nonlinear dimensionality
reduction (NLDR) technique. It is a Manifold Learning technique that does not rely
on projections like the previous algorithms. In a nutshell, LLE works by first measur‐
ing how each training instance linearly relates to its closest neighbors (c.n.), and then
looking for a low-dimensional representation of the training set where these local
relationships are best preserved (more details shortly). This makes it particularly
good at unrolling twisted manifolds, especially when there is not too much noise

The resulting 2D dataset is shown in Figure 8-12. As you can
see, the Swiss roll is completely unrolled and the distances between instances are
locally well preserved. However, distances are not preserved on a larger scale: the left
part of the unrolled Swiss roll is squeezed, while the right part is stretched. Neverthe‐
less, LLE did a pretty good job at modeling the manifold.

## Other Dimensionality Reduction Techniques


• Multidimensional Scaling (MDS) reduces dimensionality while trying to preserve
the distances between the instances (see Figure 8-13).

• Isomap creates a graph by connecting each instance to its nearest neighbors, then
reduces dimensionality while trying to preserve the geodesic distances9 between
the instances.


• t-Distributed Stochastic Neighbor Embedding (t-SNE) reduces dimensionality
while trying to keep similar instances close and dissimilar instances apart. It is
mostly used for visualization, in particular to visualize clusters of instances in
high-dimensional space (e.g., to visualize the MNIST images in 2D).

• Linear Discriminant Analysis (LDA) is actually a classification algorithm, but dur‐
ing training it learns the most discriminative axes between the classes, and these
axes can then be used to define a hyperplane onto which to project the data. The
benefit is that the projection will keep classes as far apart as possible, so LDA is a
good technique to reduce dimensionality before running another classification
algorithm such as an SVM classifier.